# Evaluating your first agent

In this tutorial, we'll build upon the web search agent from [my_first_agent.ipynb](https://github.com/mozilla-ai/any-agent/blob/main/docs/cookbook/your_first_agent.ipynb) and demonstrate how to evaluate its performance using any-agent's evaluation framework. We'll explore different evaluation methods including custom code evaluation, an LLM-based judge, and an agent-based judge. 

Note: Since we are building on the previous notebook, we encourage you to run that one first to read through details and choices available while building the agent before evaluating it.


## Install Dependencies

any-agent uses the python asyncio module to support async functionality. When running in Jupyter notebooks, this means we need to enable the use of nested event loops. We'll install any-agent and enable this below using nest_asyncio.

In [1]:
import nest_asyncio

nest_asyncio.apply()

## Set Up the Web Search Agent

First, let's recreate the web search agent from [the previous tutorial](https://github.com/mozilla-ai/any-agent/blob/main/docs/cookbook/your_first_agent.ipynb) so we have something to evaluate.

In [2]:
import os
from getpass import getpass

if "MISTRAL_API_KEY" not in os.environ:
    print("MISTRAL_API_KEY not found in environment!")
    api_key = getpass("Please enter your MISTRAL_API_KEY: ")
    os.environ["MISTRAL_API_KEY"] = api_key
    print("MISTRAL_API_KEY set for this session!")
else:
    print("MISTRAL_API_KEY found in environment.")

MISTRAL_API_KEY found in environment.


In [3]:
from any_agent import AgentConfig, AnyAgent
from any_agent.tools import search_tavily, visit_webpage

agent = AnyAgent.create(
    "tinyagent",  # See all options in https://mozilla-ai.github.io/any-agent/
    AgentConfig(
        model_id="mistral/mistral-small-latest", tools=[search_tavily, visit_webpage]
    ),
)

/home/daavoo/any-agent/.venv/lib/python3.13/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.29.3 is exactly one major version older than the runtime version 6.31.1 at a2a.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/home/daavoo/any-agent/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Run the Agent to Generate a Trace

Now let's run our agent on a test query to generate a trace that we can evaluate.

In [4]:
prompt = """What film won a Goya Award for best film in 2024?
Please provide the name of the film, the genre, a very brief
description of the film - and rotten tomatoes popcornmeter
score."""

agent_trace = agent.run(prompt)

╭──────────────────────────────────── CALL_LLM: mistral/mistral-small-latest ─────────────────────────────────────╮
│ ╭─ INPUT ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ [                                                                                                           │ │
│ │   {                                                                                                         │ │
│ │     "role": "system",                                                                                       │ │
│ │     "content": "You are an agent - please keep going until the user's query is completely resolved, before  │ │
│ │   },                                                                                                        │ │
│ │   {                                                                                                         │ │
│ │     "role": "user",                                                                                         │ │
│ │     "content": "What film won a Goya Award for best film in 2024?\nPlease provide the name of the film, the │ │
│ │   }                                                                                                         │ │
│ │ ]                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ [                                                                                                           │ │
│ │   {                                                                                                         │ │
│ │     "tool.name": "search_tavily",                                                                           │ │
│ │     "tool.args": "{\"query\": \"Goya Award for best film in 2024\"}"                                        │ │
│ │   }                                                                                                         │ │
│ │ ]                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ USAGE ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "input_tokens": 621,                                                                                      │ │
│ │   "output_tokens": 24,                                                                                      │ │
│ │   "input_cost": 6.209999999999999e-05,                                                                      │ │
│ │   "output_cost": 7.2e-06                                                                                    │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────── EXECUTE_TOOL: search_tavily ──────────────────────────────────────────╮
│ ╭─ Input ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "query": "Goya Award for best film in 2024"                                                               │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ ]8;id=232279;https://glassworksvfx.com/news/society-of-the-snow-wins-12-goyas-including-best-special-effects\Society of the Snow wins 12 Goyas, including Best Special Effects]8;;\ “Society of the Snow” Goya Awards 2024.   │ │
│ │ Best Film - J. A. Bayona, Belén Atienza, Sandra Hermida. Best Director - J. A. Bayona. Best Special Effects │ │
│ │ - Laura                                                                                                     │ │
│ │                                                                                                             │ │
│ │ ]8;id=464596;https://en.wikipedia.org/wiki/Goya_Award_for_Best_Film\Goya Award for Best Film - Wikipedia]8;;\ 2024 (39th) · Undercover, La infiltrada ; 2024 (39th) · The Blue Star, │ │
│ │ La estrella azul ; 2024 (39th) · A House on Fire, Casa en flames ; 2024 (39th) · Saturn Return              │ │
│ │                                                                                                             │ │
│ │ ]8;id=328484;https://www.imdb.com/event/ev0000299/2024/1/\Goya Awards (2024) - IMDb]8;;\ GOYA AWARDS · Goya · Best Adapted Screenplay (Mejor Guión Adaptado) · Robot       │ │
│ │ Dreams · Society of the Snow · The Teacher Who Promised the Sea · Un Amor · Jokes &                         │ │
│ │                                                                                                             │ │
│ │ ]8;id=990421;https://deadline.com/2024/02/goya-awards-complete-winners-list-1235821487/\Goya Awards Winners: 'The Society Of The Snow' Takes Best ...]8;;\ Goya Awards Complete Winners List: 'The       │ │
│ │ Society Of The Snow' Takes Best Picture & Director; Sigourney Weaver Honored With International Goya.       │ │
│ │                                                                                                             │ │
│ │ ]8;id=946731;https://en.wikipedia.org/wiki/38th_Goya_Awards\38th Goya Awards - Wikipedia]8;;\                                                                                │ │
│ │                                                                                                             │ │
│ │  • Hugo Silva "Hugo Silva (actor)") – Un amor as PiterBest Supporting Actress *   Alejandro Marín – Love &  │ │
│ │    RevolutionBest Animated Film | Awards | * Film * Director * Actor * Actress * Supporting Actor *         │ │
│ │    Supporting Actress * Original Screenplay * Adapted Screenplay * New Actor * New Actress * Ibero-American │ │
│ │    Film * European Film * New Director * Animated Film * Cinematography * Editing * Art Direction *         │ │
│ │    Production Supervision * Sound * Special Effects * Costume Design * Makeup and Hairstyles * Original     │ │
│ │    Score * Original Song * Fictional Short Film * Animated Short Film * Documentary Film * Documentary      │ │
│ │    Short Film * Honorary * International |                                                                  │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰───────────────────────────────────────────────────────────────────────────────────────────

╭──────────────────────────────────── CALL_LLM: mistral/mistral-small-latest ─────────────────────────────────────╮
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ [                                                                                                           │ │
│ │   {                                                                                                         │ │
│ │     "tool.name": "search_tavily",                                                                           │ │
│ │     "tool.args": "{\"query\": \"Society of the Snow film genre\"}"                                          │ │
│ │   },                                                                                                        │ │
│ │   {                                                                                                         │ │
│ │     "tool.name": "search_tavily",                                                                           │ │
│ │     "tool.args": "{\"query\": \"Society of the Snow film description\"}"                                    │ │
│ │   },                                                                                                        │ │
│ │   {                                                                                                         │ │
│ │     "tool.name": "search_tavily",                                                                           │ │
│ │     "tool.args": "{\"query\": \"Society of the Snow film rotten tomatoes popcornmeter score\"}"             │ │
│ │   }                                                                                                         │ │
│ │ ]                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ USAGE ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "input_tokens": 1193,                                                                                     │ │
│ │   "output_tokens": 61,                                                                                      │ │
│ │   "input_cost": 0.0001193,                                                                                  │ │
│ │   "output_cost": 1.8299999999999998e-05                                                                     │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────── EXECUTE_TOOL: search_tavily ──────────────────────────────────────────╮
│ ╭─ Input ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "query": "Society of the Snow film genre"                                                                 │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ ]8;id=305744;https://en.wikipedia.org/wiki/Society_of_the_Snow\Society of the Snow - Wikipedia]8;;\ Society of the Snow is a 2023 survival drama film directed by J. A. Bayona  │ │
│ │ and based on Pablo Vierci's 2008 book of the same name, which details the true                              │ │
│ │                                                                                                             │ │
│ │ ]8;id=437419;https://www.reelviews.net/reelviews/society-of-snow-the\Society of the Snow | Reelviews Movie Reviews]8;;\ Society of the Snow is first and foremost a man-versus-nature │ │
│ │ tale about the improbable survival of 16 people who outlast the elements for 72 days.                       │ │
│ │                                                                                                             │ │
│ │ ]8;id=852474;https://www.rottentomatoes.com/m/society_of_the_snow\Society of the Snow | Rotten Tomatoes]8;;\ R, Released Dec 22, 2023, 2h 24m, Drama ... An outstanding recreation │ │
│ │ of a grueling true story, Society of the Snow isn't the first film to dramatize these events                │ │
│ │                                                                                                             │ │
│ │ ]8;id=51879;https://www.imdb.com/title/tt16277242/\Society of the Snow (2023) - IMDb]8;;\ And at my lowest points, when I thought all was lost, this is the story   │ │
│ │ that would creep into my mind, and I would imagine (there's that word again: IMAGINE) what it must've been  │ │
│ │ like to be on the side of that mountain, on that glacier, in that real-life scenario having to eat my dead  │ │
│ │ friends as I freeze, knowing I'm starving to death, watching my brothers starve to death, dying from their  │ │
│ │ injuries, freezing 13,000' up in the Andes and no one is looking for you, you know the search has been      │ │
│ │ called off. A. Bayona delivers the film 'Society of the Snow' and I no longer have to try to imagine, as    │ │
│ │ I've now seen it, on film, on a big theater screen.                                                         │ │
│ │                                                                                                             │ │
│ │ ]8;id=520624;https://letterboxd.com/film/society-of-the-snow/\Society of the Snow (2023) directed by J.A. Bayona - Letterboxd]8;;\ On October 13, 1972, Uruguayan Air Force    │ │
│ │ Flight 571, chartered to take a rugby team to Chile, crashes into a glacier in the heart of the Andes.      │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────── EXECUTE_TOOL: search_tavily ──────────────────────────────────────────╮
│ ╭─ Input ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "query": "Society of the Snow film description"                                                           │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ ]8;id=974131;https://www.browndailyherald.com/article/2024/01/society-of-the-snow-offers-a-deeper-emotional-take-on-survival-story\'Society of the Snow' offers a deeper, emotional take on survival story]8;;\ “Society of the Snow” is an intense │ │
│ │ and heartbreaking film that viscerally explores the limits of humanity under extreme circumstances.         │ │
│ │                                                                                                             │ │
│ │ ]8;id=571456;https://www.netflix.com/tudum/articles/society-of-the-snow-release-date-news\Society of the Snow: Who's in the Cast of the True Story? - Netflix]8;;\ What happens in Society of the Snow?    │ │
│ │ The film depicts the real-life events of the crash of Flight 571 — from the day Uruguay's Old Christians    │ │
│ │ Club rugby team left for a match in Santiago, Chile, to 72 days later when 16 survivors finally came home.  │ │
│ │ It's told primarily through rugby player Numa Turcatti's point of view.                                     │ │
│ │                                                                                                             │ │
│ │ ]8;id=200689;https://www.imdb.com/title/tt16277242/\Society of the Snow (2023) - IMDb]8;;\ And at my lowest points, when I thought all was lost, this is the story   │ │
│ │ that would creep into my mind, and I would imagine (there's that word again: IMAGINE) what it must've been  │ │
│ │ like to be on the side of that mountain, on that glacier, in that real-life scenario having to eat my dead  │ │
│ │ friends as I freeze, knowing I'm starving to death, watching my brothers starve to death, dying from their  │ │
│ │ injuries, freezing 13,000' up in the Andes and no one is looking for you, you know the search has been      │ │
│ │ called off. A. Bayona delivers the film 'Society of the Snow' and I no longer have to try to imagine, as    │ │
│ │ I've now seen it, on film, on a big theater screen.                                                         │ │
│ │                                                                                                             │ │
│ │ ]8;id=687502;https://www.rottentomatoes.com/m/society_of_the_snow\Society of the Snow | Rotten Tomatoes]8;;\ An outstanding recreation of a grueling true story, Society of the    │ │
│ │ Snow isn't the first film to dramatize these events, but it might be the best.                              │ │
│ │                                                                                                             │ │
│ │ ]8;id=877077;https://en.wikipedia.org/wiki/Society_of_the_Snow\Society of the Snow - Wikipedia]8;;\ The true story of a Uruguayan rugby team's experience in 1972 after         │ │
│ │ Uruguayan Air Force Flight 571 crashed in the Andes Mountains.                                              │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────── EXECUTE_TOOL: search_tavily ──────────────────────────────────────────╮
│ ╭─ Input ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "query": "Society of the Snow film rotten tomatoes popcornmeter score"                                    │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ ]8;id=552542;https://www.rottentomatoes.com/m/society_of_the_snow\Society of the Snow | Rotten Tomatoes]8;;\ Discover reviews, ratings, and trailers for Society of the Snow on    │ │
│ │ Rotten Tomatoes ... Fresh audience score. - - 88% Popcornmeter 1,000+ Ratings. In 1972,                     │ │
│ │                                                                                                             │ │
│ │ ]8;id=687833;https://www.rottentomatoes.com/m/society_of_the_snow/reviews\Society of the Snow | Reviews - Rotten Tomatoes]8;;\ Explore critic and audience reviews for Society of the Snow │ │
│ │ on Rotten Tomatoes. Uncover scores, opinions, and ratings.                                                  │ │
│ │                                                                                                             │ │
│ │ ]8;id=407756;https://www.reddit.com/r/movies/comments/1dwcpxs/is_rotten_tomatoes_basically_worthless_for/\Is Rotten Tomatoes basically worthless for telling good movies from ...]8;;\ : r/movies : r/movies Image 1:      │ │
│ │ r/movies icon Go to movies r/movies Image 3: r/movies iconr/movies But going by the actual critic scores,   │ │
│ │ it's a 7/10 movie at best. For another extreme example, a movie could get 50% 10/10 ratings and 50% 0/10    │ │
│ │ ratings. Top Posts *   Games  *   Gaming News & Discussion *   Other Games *   Movies & TV  *   Action      │ │
│ │ Movies & Series *   Animated Movies & Series *   Comedy Movies & Series *   Documentary Movies & Series *   │ │
│ │ Drama Movies & Series *   Fantasy Movies & Series *   Horror Movies & Series *   Movie News & Discussion *  │ │
│ │ Romance Movies & Series *   Sci-Fi Movies & Series *   Superhero Movies & Series *   About Reddit *   Best  │ │
│ │ of Reddit                                                                                                   │ │
│ │                                                                                                             │ │
│ │ ]8;id=883134;https://www.rottentomatoes.com/critics/silvia-mariscal/movies\Silvia Mariscal Movie Reviews & Previews - Rotten Tomatoes]8;;\ Read Movie and TV reviews from Silvia Mariscal   │ │
│ │ on Rotten Tomatoes, where critics reviews are aggregated to tally a Certified Fresh, Fresh or Rotten        │ │
│ │                                                                                                             │ │
│ │ ]8;id=319668;https://editorial.rottentomatoes.com/guide/best-movies-of-2023/\Best Movies of 2023: Every Certified Fresh Movie of the Year]8;;\ Critics Consensus: An understated yet          │ │
│ │ impactful story of female empowerment, Shayda establishes first-time feature filmmaker Noora Niasari as a   │ │
│ │ remarkably assured talent. Synopsis: From acclaimed director Alexander Payne, THE HOLDOVERS follows a       │ │
│ │ curmudgeonly instructor (Paul Giamatti) at a New England prep school who   Critics Consensus: A smart,      │ │
│ │ solidly crafted procedural that's anchored in family drama, Anatomy of a Fall finds star Sandra Hüller and  │ │
│ │ director/co-writer Justine Triet operating at peak power. Synopsis: From 

╭──────────────────────────────────── CALL_LLM: mistral/mistral-small-latest ─────────────────────────────────────╮
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ [                                                                                                           │ │
│ │   {                                                                                                         │ │
│ │     "tool.name": "final_answer",                                                                            │ │
│ │     "tool.args": "{\"answer\": \"The film that won the Goya Award for best film in 2024 is 'Society of the  │ │
│ │   }                                                                                                         │ │
│ │ ]                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ USAGE ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "input_tokens": 2987,                                                                                     │ │
│ │   "output_tokens": 186,                                                                                     │ │
│ │   "input_cost": 0.0002987,                                                                                  │ │
│ │   "output_cost": 5.5799999999999994e-05                                                                     │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────── EXECUTE_TOOL: final_answer ───────────────────────────────────────────╮
│ ╭─ Input ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "answer": "The film that won the Goya Award for best film in 2024 is 'Society of the Snow'. It is a survi │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ The film that won the Goya Award for best film in 2024 is 'Society of the Snow'. It is a survival drama     │ │
│ │ film directed by J. A. Bayona and based on Pablo Vierci's 2008 book of the same name, which details the     │ │
│ │ true story of a Uruguayan rugby team's experience in 1972 after Uruguayan Air Force Flight 571 crashed in   │ │
│ │ the Andes Mountains. The film depicts the real-life events of the crash of Flight 571 — from the day        │ │
│ │ Uruguay's Old Christians Club rugby team left for a match in Santiago, Chile, to 72 days later when 16      │ │
│ │ survivors finally came home. It's told primarily through rugby player Numa Turcatti's point of view. The    │ │
│ │ film has a Rotten Tomatoes popcornmeter score of 88%.                                                       │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## View the Agent Results

Let's first see what our agent produced:


In [5]:
print(agent_trace.final_output)  # Final answer
print(f"Duration: {agent_trace.duration.total_seconds():.2f} seconds")
print(f"Usage: {agent_trace.tokens.total_tokens:,}")
print(f"Cost (USD): {agent_trace.cost.total_cost:.6f}")

The film that won the Goya Award for best film in 2024 is 'Society of the Snow'. It is a survival drama film directed by J. A. Bayona and based on Pablo Vierci's 2008 book of the same name, which details the true story of a Uruguayan rugby team's experience in 1972 after Uruguayan Air Force Flight 571 crashed in the Andes Mountains. The film depicts the real-life events of the crash of Flight 571 — from the day Uruguay's Old Christians Club rugby team left for a match in Santiago, Chile, to 72 days later when 16 survivors finally came home. It's told primarily through rugby player Numa Turcatti's point of view. The film has a Rotten Tomatoes popcornmeter score of 88%.
Duration: 23.24 seconds
Usage: 5,072
Cost (USD): 0.000561


## Method 1: Custom Code Evaluation

Before using LLM-based evaluation, let's start with deterministic custom code evaluation. This is often more efficient, reliable, and cost-effective for specific criteria.

Some criteria are clearly quantitative: a result exists or it doesn't, it has a measurable length, the number of steps can be counted and a tool was either called or wasn't.

In [6]:
from any_agent.tracing.agent_trace import AgentTrace
from any_agent.tracing.attributes import GenAI


def check_tool_usage(trace: AgentTrace, required_tool: str) -> bool:
    """Check if a specific tool was used in the trace."""
    return any(
        span.attributes[GenAI.TOOL_NAME] == required_tool
        for span in trace.spans
        if span.is_tool_execution()
    )


def evaluate_web_search_efficiency(trace: AgentTrace) -> dict:
    """Custom evaluation function for web search agent efficiency criteria."""
    # Direct access to trace properties
    token_count = trace.tokens.total_tokens
    step_count = len(trace.spans)
    final_output = trace.final_output
    duration = trace.duration.total_seconds()
    # Check if web search tools were used
    used_search = check_tool_usage(trace, "search_tavily")
    used_visit = check_tool_usage(trace, "visit_webpage")
    # Apply quantitative criteria
    results = {
        "token_efficient": token_count
        < 20000,  # Magic number alert: adjust to what you consider reasonable for your budget
        "step_efficient": step_count
        <= 10,  # A high number of steps would point at problems, but this is also a debatable limit
        "has_output": final_output is not None and len(str(final_output)) > 5,
        "short_output": len(str(final_output)) < 10 if final_output else 0,
        "used_web_search": used_search,
        "used_webpage_visit": used_visit,
        "reasonable_duration": duration < 60,
    }
    # Choose the quantitative criteria you care most about
    results["passed"] = all(
        [
            results["token_efficient"],
            results["step_efficient"],
            results["has_output"],
            results["used_web_search"],
            results["short_output"],
        ]
    )
    return results

In [7]:
evaluation = evaluate_web_search_efficiency(agent_trace)
print("Custom Code Evaluation Results:")
for key, value in evaluation.items():
    print(f"  {key}: {value}")

Custom Code Evaluation Results:
  token_efficient: True
  step_efficient: True
  has_output: True
  short_output: False
  used_web_search: True
  used_webpage_visit: False
  reasonable_duration: True
  passed: False


## Method 2: LLM Judge Evaluation

The method above is already useful and can assess quantitative results (how long or how costly answers were, whether a specific tool was present). Programmatic evaluations are less costly, more deterministic, but also less flexible. They can see that a tool was used: but was the result well understood? Was the content actually used to extract an answer? This is a qualitative assessment.

For such criteria, you can use the `LlmJudge`. This is great for evaluating response quality, helpfulness, and other subjective criteria.

### 💡 Good to know: different models

Notice we use a different LLM as a judge to the one we used for the original agent, as LLM judges are known to have a [bias towards their own results](https://neurips.cc/virtual/2024/poster/96672).

In [8]:
from any_agent.evaluation import LlmJudge

# Create an LLM judge
judge = LlmJudge(model_id="gpt-4.1-mini")

# Define evaluation questions - notice the last one is not like the others
evaluation_questions = [
    "Did the agent provide a clear and concise answer?",
    "Did the agent correctly identify the genre?",
    "Did the agent provide a brief description (under 10 words) of the film?",
]

# Run evaluations
print("LLM Judge Evaluation Results:")
print("=" * 60)

results = []
for i, question in enumerate(evaluation_questions, 1):
    result = judge.run(context=str(agent_trace.spans_to_messages()), question=question)
    results.append(result)
    print(f"Question {i}: {question}")
    print(f"  Passed: {result.passed}")
    print(f"  Reasoning: {result.reasoning}")
    print("-" * 60)

# Summary
passed_count = sum(1 for r in results if r.passed)
print(f"\nOverall: {passed_count}/{len(results)} criteria passed")

LLM Judge Evaluation Results:


Question 1: Did the agent provide a clear and concise answer?
  Passed: True
  Reasoning: The agent clearly identified the film that won the Goya Award for best film in 2024 as 'Society of the Snow.' The response included the film's genre (survival drama), a very brief description of the film summarizing its plot and real-life basis, and the Rotten Tomatoes popcornmeter score of 88%. The answer was concise, directly addressing all parts of the user's query without extraneous information.
------------------------------------------------------------
Question 2: Did the agent correctly identify the genre?
  Passed: True
  Reasoning: The agent identified the genre of the film 'Society of the Snow' as a survival drama, which is supported by multiple sources in the context. The Wikipedia summary, Reelviews movie review, and other sources all confirm it as a survival drama film. Therefore, the agent correctly identified the genre.
------------------------------------------------------------
Q

### 💡 Good to know: fuzzy criteria 

Notice Question 3: if you run the evaluation multiple times, it won't pass or fail consistently, since the LLM judge may interpret that only the description should be under 10 words, not necessarily the whole Agent's answer. In the programmatic method, there is nothing to interpret: we check that the final output was under 10 words. 

This showcases the main downside with using an LLM judge: as with humans, criteria can be misunderstood. 

On the other hand, using a programmatic approach to assess clarity, for example, would have been rather complex without an LLM judge.

A take-home message here is to use custom code when criteria can be counted or measured, and think of using an LLMJudge when your criteria are qualitative.

## Method 3: Agent Judge Evaluation

For more complex evaluations that require inspecting specific aspects of the trace, we can use the `AgentJudge`. Notice the AgentJudge can:

* call built-in tools to get straight to relevant parts of the traces (e.g. final output),
* call additional tools that the original agent did not have. For example, you will see below how we give it a second search tool so it can do its own research to check if the original agent's answer was correct.

As with the LLMJudge, we choose a different model to the one enabling the original judge.

Notice, if you do not have a [Tavily API key](https://www.tavily.com/), you can import and use `search_web` (Duck Duck Go Search).

In [9]:
from any_agent.evaluation import AgentJudge
from any_agent.tools import search_tavily

# Create an agent judge
agent_judge = AgentJudge(model_id="gpt-4.1-mini")

# Define a complex evaluation question that requires trace inspection
complex_question = """
Evaluate the agent's performance on this web search task by verifying
whether the agent correctly used web search to find relevant information
for the winner film of the Goya Award in 2024 and its Rotten Tomatoes rating?

Use the available tools to inspect the trace and, specially, make sure
the agent visited Rotten Tomatoes and checked the audience score, not
the critics score.
"""

# Run the agent judge evaluation
eval_trace = agent_judge.run(
    trace=agent_trace,
    question=complex_question,
    additional_tools=[
        search_tavily
    ],  # Give the judge access to web search for verification
)

# Get the evaluation result
result = eval_trace.final_output
print("Agent Judge Evaluation Result:")
print("=" * 60)
print(f"Passed: {result.passed}")
print(f"Reasoning: {result.reasoning}")
print("=" * 60)

╭──────────────────────────────────────────── CALL_LLM: gpt-4.1-mini ─────────────────────────────────────────────╮
│ ╭─ INPUT ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ [                                                                                                           │ │
│ │   {                                                                                                         │ │
│ │     "role": "system",                                                                                       │ │
│ │     "content": "\nYou are a helpful assistant that will be used to evaluate the correctness of an agent tra │ │
│ │   },                                                                                                        │ │
│ │   {                                                                                                         │ │
│ │     "role": "user",                                                                                         │ │
│ │     "content": "\nEvaluate the agent's performance on this web search task by verifying\nwhether the agent  │ │
│ │   }                                                                                                         │ │
│ │ ]                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ [                                                                                                           │ │
│ │   {                                                                                                         │ │
│ │     "tool.name": "get_final_output",                                                                        │ │
│ │     "tool.args": "{}"                                                                                       │ │
│ │   },                                                                                                        │ │
│ │   {                                                                                                         │ │
│ │     "tool.name": "get_messages_from_trace",                                                                 │ │
│ │     "tool.args": "{}"                                                                                       │ │
│ │   }                                                                                                         │ │
│ │ ]                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ USAGE ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "input_tokens": 532,                                                                                      │ │
│ │   "output_tokens": 42,                                                                                      │ │
│ │   "input_cost": 0.0002128,                                                                                  │ │
│ │   "output_cost": 6.72e-05                                                                                   │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────── EXECUTE_TOOL: get_final_output ─────────────────────────────────────────╮
│ ╭─ Input ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {}                                                                                                          │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ The film that won the Goya Award for best film in 2024 is 'Society of the Snow'. It is a survival drama     │ │
│ │ film directed by J. A. Bayona and based on Pablo Vierci's 2008 book of the same name, which details the     │ │
│ │ true story of a Uruguayan rugby team's experience in 1972 after Uruguayan Air Force Flight 571 crashed in   │ │
│ │ the Andes Mountains. The film depicts the real-life events of the crash of Flight 571 — from the day        │ │
│ │ Uruguay's Old Christians Club rugby team left for a match in Santiago, Chile, to 72 days later when 16      │ │
│ │ survivors finally came home. It's told primarily through rugby player Numa Turcatti's point of view. The    │ │
│ │ film has a Rotten Tomatoes popcornmeter score of 88%.                                                       │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────── EXECUTE_TOOL: get_messages_from_trace ─────────────────────────────────────╮
│ ╭─ Input ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {}                                                                                                          │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │                                                   system                                                    │ │
│ │                                                                                                             │ │
│ │ You are an agent - please keep going until the user's query is completely resolved, before ending your turn │ │
│ │ and yielding back to the user. Only terminate your turn when you are sure that the problem is solved, or if │ │
│ │ you need more info from the user to solve the problem.                                                      │ │
│ │                                                                                                             │ │
│ │ If you are not sure about anything pertaining to the user's request, use your tools to read files and       │ │
│ │ gather the relevant information: do NOT guess or make up an answer.                                         │ │
│ │                                                                                                             │ │
│ │ You MUST plan extensively before each function call, and reflect extensively on the outcomes of the         │ │
│ │ previous function calls. DO NOT do this entire process by making function calls only, as this can impair    │ │
│ │ your ability to solve the problem and think insightfully.                                                   │ │
│ │                                                                                                             │ │
│ │                                                    user                                                     │ │
│ │                                                                                                             │ │
│ │ What film won a Goya Award for best film in 2024? Please provide the name of the film, the genre, a very    │ │
│ │ brief description of the film - and rotten tomatoes popcornmeter score.                                     │ │
│ │                                                                                                             │ │
│ │                                                  assistant                                                  │ │
│ │                                                                                                             │ │
│ │ [{"tool.name": "search_tavily", "tool.args": "{"query": "Goya Award for best film in 2024"}"}]              │ │
│ │                                                                                                             │ │
│ │                                                  assistant                                                  │ │
│ │                                                                                                             │ │
│ │ [Tool search_tavily executed: ]8;id=132425;https://glassworksvfx.com/news/society-of-the-snow-wins-12-goyas-including-best-special-effects\Society of the Snow wins 12 Goyas, including Best Special Effects]8;;\ “Society of │ │
│ │ the Snow” Goya Awards 2024. Best Film - J. A. Bayona, Belén Atienza, Sandra Hermida. Best Director - J. A.  │ │
│ │ Bayona. Best Special Effects - Laura                                                                        │ │
│ │                                                                                                             │ │
│ │ ]8;id=717759;https://en.wikipedia.org/wiki/Goya_A

╭──────────────────────────────────────────── CALL_LLM: gpt-4.1-mini ─────────────────────────────────────────────╮
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ [                                                                                                           │ │
│ │   {                                                                                                         │ │
│ │     "tool.name": "get_messages_from_trace",                                                                 │ │
│ │     "tool.args": "{}"                                                                                       │ │
│ │   }                                                                                                         │ │
│ │ ]                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ USAGE ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "input_tokens": 3976,                                                                                     │ │
│ │   "output_tokens": 12,                                                                                      │ │
│ │   "input_cost": 0.0015903999999999998,                                                                      │ │
│ │   "output_cost": 1.92e-05                                                                                   │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────── EXECUTE_TOOL: get_messages_from_trace ─────────────────────────────────────╮
│ ╭─ Input ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {}                                                                                                          │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │                                                   system                                                    │ │
│ │                                                                                                             │ │
│ │ You are an agent - please keep going until the user's query is completely resolved, before ending your turn │ │
│ │ and yielding back to the user. Only terminate your turn when you are sure that the problem is solved, or if │ │
│ │ you need more info from the user to solve the problem.                                                      │ │
│ │                                                                                                             │ │
│ │ If you are not sure about anything pertaining to the user's request, use your tools to read files and       │ │
│ │ gather the relevant information: do NOT guess or make up an answer.                                         │ │
│ │                                                                                                             │ │
│ │ You MUST plan extensively before each function call, and reflect extensively on the outcomes of the         │ │
│ │ previous function calls. DO NOT do this entire process by making function calls only, as this can impair    │ │
│ │ your ability to solve the problem and think insightfully.                                                   │ │
│ │                                                                                                             │ │
│ │                                                    user                                                     │ │
│ │                                                                                                             │ │
│ │ What film won a Goya Award for best film in 2024? Please provide the name of the film, the genre, a very    │ │
│ │ brief description of the film - and rotten tomatoes popcornmeter score.                                     │ │
│ │                                                                                                             │ │
│ │                                                  assistant                                                  │ │
│ │                                                                                                             │ │
│ │ [{"tool.name": "search_tavily", "tool.args": "{"query": "Goya Award for best film in 2024"}"}]              │ │
│ │                                                                                                             │ │
│ │                                                  assistant                                                  │ │
│ │                                                                                                             │ │
│ │ [Tool search_tavily executed: ]8;id=44055;https://glassworksvfx.com/news/society-of-the-snow-wins-12-goyas-including-best-special-effects\Society of the Snow wins 12 Goyas, including Best Special Effects]8;;\ “Society of │ │
│ │ the Snow” Goya Awards 2024. Best Film - J. A. Bayona, Belén Atienza, Sandra Hermida. Best Director - J. A.  │ │
│ │ Bayona. Best Special Effects - Laura                                                                        │ │
│ │                                                                                                             │ │
│ │ ]8;id=118371;https://en.wikipedia.org/wiki/Goya_Aw

...

╭───────────────────────────────────── EXECUTE_TOOL: get_messages_from_trace ─────────────────────────────────────╮
│ ╭─ Input ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {}                                                                                                          │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │                                                   system                                                    │ │
│ │                                                                                                             │ │
│ │ You are an agent - please keep going until the user's query is completely resolved, before ending your turn │ │
│ │ and yielding back to the user. Only terminate your turn when you are sure that the problem is solved, or if │ │
│ │ you need more info from the user to solve the problem.                                                      │ │
│ │                                                                                                             │ │
│ │ If you are not sure about anything pertaining to the user's request, use your tools to read files and       │ │
│ │ gather the relevant information: do NOT guess or make up an answer.                                         │ │
│ │                                                                                                             │ │
│ │ You MUST plan extensively before each function call, and reflect extensively on the outcomes of the         │ │
│ │ previous function calls. DO NOT do this entire process by making function calls only, as this can impair    │ │
│ │ your ability to solve the problem and think insightfully.                                                   │ │
│ │                                                                                                             │ │
│ │                                                    user                                                     │ │
│ │                                                                                                             │ │
│ │ What film won a Goya Award for best film in 2024? Please provide the name of the film, the genre, a very    │ │
│ │ brief description of the film - and rotten tomatoes popcornmeter score.                                     │ │
│ │                                                                                                             │ │
│ │                                                  assistant                                                  │ │
│ │                                                                                                             │ │
│ │ [{"tool.name": "search_tavily", "tool.args": "{"query": "Goya Award for best film in 2024"}"}]              │ │
│ │                                                                                                             │ │
│ │                                                  assistant                                                  │ │
│ │                                                                                                             │ │
│ │ [Tool search_tavily executed: ]8;id=816824;https://glassworksvfx.com/news/society-of-the-snow-wins-12-goyas-including-best-special-effects\Society of the Snow wins 12 Goyas, including Best Special Effects]8;;\ “Society of │ │
│ │ the Snow” Goya Awards 2024. Best Film - J. A. Bayona, Belén Atienza, Sandra Hermida. Best Director - J. A.  │ │
│ │ Bayona. Best Special Effects - Laura                                                                        │ │
│ │                                                                                                             │ │
│ │ ]8;id=737994;https://en.wikipedia.org/wiki/Goya_A

╭──────────────────────────────────────────── CALL_LLM: gpt-4.1-mini ─────────────────────────────────────────────╮
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ [                                                                                                           │ │
│ │   {                                                                                                         │ │
│ │     "tool.name": "final_answer",                                                                            │ │
│ │     "tool.args": "{\"answer\":\"passed: true; reasoning: The agent correctly identified the winner film of  │ │
│ │   }                                                                                                         │ │
│ │ ]                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ USAGE ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "input_tokens": 19176,                                                                                    │ │
│ │   "output_tokens": 88,                                                                                      │ │
│ │   "input_cost": 0.0076704,                                                                                  │ │
│ │   "output_cost": 0.0001408                                                                                  │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────── EXECUTE_TOOL: final_answer ───────────────────────────────────────────╮
│ ╭─ Input ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "answer": "passed: true; reasoning: The agent correctly identified the winner film of the Goya Award in 2 │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ passed: true; reasoning: The agent correctly identified the winner film of the Goya Award in 2024 as        │ │
│ │ 'Society of the Snow' and provided the genre, description, and Rotten Tomatoes popcornmeter score of 88%.   │ │
│ │ The agent visited Rotten Tomatoes and specifically cited the audience score (popcornmeter score), not the   │ │
│ │ critics score, as required by the task.                                                                     │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── CALL_LLM: gpt-4.1-mini ─────────────────────────────────────────────╮
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "passed": true,                                                                                           │ │
│ │   "reasoning": "The agent correctly identified the winner film of the Goya Award in 2024 as 'Society of the │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ USAGE ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "input_tokens": 218,                                                                                      │ │
│ │   "output_tokens": 78,                                                                                      │ │
│ │   "input_cost": 8.719999999999999e-05,                                                                      │ │
│ │   "output_cost": 0.0001248                                                                                  │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Agent Judge Evaluation Result:
Passed: True
Reasoning: The agent correctly identified the winner film of the Goya Award in 2024 as 'Society of the Snow' and provided the genre, description, and Rotten Tomatoes popcornmeter score of 88%. The agent visited Rotten Tomatoes and specifically cited the audience score (popcornmeter score), not the critics score, as required by the task.


Notice how giving the judge tools enables it to check **independently** whether the original agent successfully did its job.